In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding
from keras.layers import Input
from keras.layers.merge import Concatenate
from tensorflow.keras.utils import to_categorical
from keras.models import Model
import matplotlib.pyplot as plt
import re
import string
import numpy as np
import pandas as pd
import os
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm_notebook

In [28]:
def read_text_file(file_path):
    with open(file_path, 'r',encoding="UTF-8") as f:
        return f.read()
    
training_text = ""
# iterate through all files
path = "/content/drive/MyDrive/Colab Notebooks/NextWordPrediction/Data/"
max_file = 7
count = 0
for file in os.listdir(path):
    count+=1
    if count>max_file:
        break
    print(file)
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = path+file
        training_text+=read_text_file(file_path)

Book 3 - The Prisoner of Azkaban.txt
Book 4 - The Goblet of Fire.txt
Book 7 - The Deathly Hallows.txt
Book 2 - The Chamber of Secrets.txt
Book 5 - The Order of the Phoenix.txt
Book 6 - The Half Blood Prince.txt
Book 1 - The Philosopher's Stone.txt


In [29]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [32]:
filtered_training_data = " ".join([w for w in training_text.split() if not w.lower() in stopwords])

In [33]:
filtered_training_data[0:100]

'/ OWL POST Harry Potter highly unusual boy many ways. one thing, hated summer holidays time year. an'

In [35]:
from nltk.tokenize import sent_tokenize
filtered_training_data = re.sub(r'\W+', ' ', filtered_training_data).lower()
filtered_training_data = sent_tokenize(filtered_training_data)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(filtered_training_data)
word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

vocab_size = len(word2id) + 1 
embed_size = 100

wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in filtered_training_data]
print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 22794
Vocabulary Sample: [('harry', 1), ('s', 2), ('said', 3), ('t', 4), ('ron', 5), ('i', 6), ('potter', 7), ('you', 8), ('it', 9), ('hermione', 10)]


In [36]:
from keras.preprocessing.sequence import skipgrams

# generate skip-grams
skip_grams = [skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in wids]

# view sample skip-grams
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          id2word[pairs[i][0]], pairs[i][0], 
          id2word[pairs[i][1]], pairs[i][1], 
          labels[i]))

(yet (235), bed (232)) -> 1
(not (185), since (381)) -> 1
(takin (6518), don (32)) -> 1
(slytherins (968), malfoy (63)) -> 1
(ludicrously (11891), withou (15008)) -> 0
(help (219), hallows (114)) -> 1
(like (22), 721harry (20468)) -> 0
(there (58), gaps (6488)) -> 0
(steps (554), hours (713)) -> 0
(said (3), perch (4378)) -> 0


In [46]:
from keras.layers.core import Dense, Reshape
from keras.layers import Concatenate
word_model = Sequential()
word_model.add(Embedding(vocab_size, embed_size,
                         embeddings_initializer="glorot_uniform",
                         input_length=1))
word_model.add(Reshape((embed_size, )))

context_model = Sequential()
context_model.add(Embedding(vocab_size, embed_size,
                  embeddings_initializer="glorot_uniform",
                  input_length=1))
context_model.add(Reshape((embed_size,)))

model = Sequential()
model.add(Concatenate([word_model, context_model]))
model.add(Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="rmsprop",metrics=['accuracy'])

# view model summary
model.summary()

ValueError: ignored

In [47]:
for epoch in range(1, 6):
    loss = 0
    for i, elem in enumerate(skip_grams):
        pair_first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        pair_second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        labels = np.array(elem[1], dtype='int32')
        X = [pair_first_elem, pair_second_elem]
        Y = labels
        if i % 10000 == 0:
            print('Processed {} (skip_first, skip_second, relevance) pairs'.format(i))
        loss += model.train_on_batch(X,Y)  

    print('Epoch:', epoch, 'Loss:', loss)

Processed 0 (skip_first, skip_second, relevance) pairs
Consider rewriting this model with the Functional API.


TypeError: ignored

In [48]:
X

[array([ 235,  185, 6518, ...,   54,  295,    8], dtype=int32),
 array([  232,   381,    32, ..., 22648,  1279,  4418], dtype=int32)]